### This script is a port of EnergyID's code that calculates a linear regression on heating data

# Imports and setup

In [ ]:
import sys
import os
import inspect
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = 12,8

from opengrid.library import houseprint

# Load Test Data

## Load Degree Days

In [ ]:
dfWeather = pd.read_csv('../data/WeatherData.csv')

In [ ]:
dfWeather = dfWeather.set_index(pd.DatetimeIndex(dfWeather['Date']))

In [ ]:
tsDegreeDays = dfWeather['DegreeDays']

In [ ]:
tsDegreeDays.name = 'x'

In [ ]:
tsDegreeDays = tsDegreeDays.resample(rule='MS',how='sum')

## Load Measurements

In [ ]:
import json
import numpy as np

In [ ]:
json_data = open('../data/diedrik.json')

In [ ]:
js = json.load(json_data)

In [ ]:
#make two arrays with data
X = [point['x']/1000 for point in js['points']]
Y = [point['y'] for point in js['points']]

In [ ]:
#convert unix timestamp to datetime
X = np.array(X).astype('datetime64[s]')

In [ ]:
tsEnergyData = pd.Series(data=Y,index=pd.DatetimeIndex(X), name='y')

### Bring weather data and energy data together

In [ ]:
data = pd.concat([tsDegreeDays,tsEnergyData],axis=1).dropna()

In [ ]:
print data

# Run Analysis

In [ ]:
from opengrid.library import linearRegression
linregress = linearRegression.LinearRegression(data)

In [ ]:
fig = linregress.to_plt()

In [ ]:
linregress2 = linearRegression.LinearRegression2(data=data, breakpoint=60)

In [ ]:
fig = linregress2.to_plt()

In [ ]:
linregress3 = linearRegression.LinearRegression3(data=data, breakpoint=60, percentage=2, include_end_of_base_load=True)

In [ ]:
fig = linregress3.to_plt()

# Plot Models

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.plot_date(data.index, data['x'],'-',label='gas')
ax1.plot_date(data.index, linregress.get_y(data['x']),'-', label='simple regression')
ax1.plot_date(data.index, linregress2.get_y(data['x']),'-', label='with baseline')
ax1.plot_date(data.index, linregress3.get_y(data['x']),'-', label='progressive baseline')
plt.legend()